<a href="https://colab.research.google.com/github/EduardoMoraesRitter/word-tree-analysis/blob/main/word_tree_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Word Tree

para mais detalhes: https://developers.google.com/chart/interactive/docs/gallery/wordtree


In [39]:
from IPython.display import HTML
from string import Template
import requests

In [41]:
#dados(frases)

json_tree = [['Frases'],
        ['cats are better than dogs'], ['dog eat kibble'], ['cats are better than hamsters'],
        ['dog are awesome'], ['cats are people too'], ['cats eat mice'],
        ['cats meowing'], ['cats in the cradle'], ['cats eat mice'],
        ['cats in the cradle lyrics'], ['cats eat kibble'], ['cats for adoption'],
        ['dog are family'], ['dog eat mice'], ['dog are better than kittens'],
        ['cats are evil'], ['cats are weird'], ['cats eat mice']]

In [42]:
#ler um aqrquivo ou a URL onde comtem o HTML do google
url = requests.get("https://raw.githubusercontent.com/EduardoMoraesRitter/word-tree-analysis/main/wordtree_template.html")

#Ler HTML do aquirvo
htmltext = Template(url.text)

#subistituir os valores ${}
htmlfinal =  htmltext.substitute(root_word="are", tree_type="double", json_formatted=json_tree)

#executar o HML
HTML(htmlfinal)

### Exemplo com função

In [5]:
def get_word_tree_html(json_formatted, root_word, lowercase=True, tree_type='double'):
    
    with open('https://raw.githubusercontent.com/EduardoMoraesRitter/word-tree-analysis/main/wordtree_template.html', 'r') as my_file:
        html_template = Template(my_file.read())
    return html_template.substitute(root_word=root_word,
                                    tree_type=tree_type,
                                    json_formatted=json_formatted.lower() if lowercase else json_formatted)

In [ ]:
json = [['Frases'],
        ['cats are better than dogs'],
        ['dog eat kibble'],
        ['cats are better than hamsters'],
        ['dog are awesome'],
        ['cats are people too'],
        ['cats eat mice'],
        ['cats meowing'],
        ['cats in the cradle'],
        ['cats eat mice'],
        ['cats in the cradle lyrics'],
        ['cats eat kibble'],
        ['cats for adoption'],
        ['dog are family'],
        ['dog eat mice'],
        ['dog are better than kittens'],
        ['cats are evil'],
        ['cats are weird'],
        ['cats eat mice']]

HTML(get_word_tree_html(json, 'are', lowercase=False, tree_type='double'))

In [ ]:
"""
<html>

<head>
    <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
    <script type="text/javascript">

        google.charts.load('current', { packages: ['wordtree'] });

        var chart;
        var data;
        var options = {
            wordtree: {
                format: 'implicit',
                word: '${root_word}',
                type: '${tree_type}'
            }
        };

        function initChart() {
            data = google.visualization.arrayToDataTable(${json_formatted});
            chart = new google.visualization.WordTree(document.getElementById('wordtree_basic'));
        }

        function drawChart() {
            chart.draw(data, options);
        }

        function setRootWordFromForm() {
            options.wordtree.word = document.getElementById('root_word').value;
            drawChart();
        }

        google.charts.setOnLoadCallback(function () {
            initChart();
            drawChart();
        });

    </script>
</head>

<body>
    Root Word:
    <input type="text" id="root_word" value="${root_word}">
    <button type="button" onclick="setRootWordFromForm()">Update Tree</button>
    <br>
    <div id="wordtree_basic" style="width: 900px; height: 600px;"></div>
</body>

</html>
"""

In [ ]:
#forma de ler o aquivo
with open('wordtree_template.html', 'r') as my_file:
  print(my_file)
  html_template = Template(my_file.read())
teste = html_template.substitute(root_word="are", tree_type="double", json_formatted=[["aa"],["are"]])
HTML(teste)

In [ ]:
#outra forma de ler um aquivo
import codecs
f=codecs.open("wordtree_template.html", 'r')
print(f.read())

# Wordcloud

### Define the helper method

In [ ]:
def generate_cloud(texts, max_words=30):
    # Add more words here if you want to ignore them:
    my_stopwords = STOPWORDS.copy()
    my_stopwords.update(['go', 'ya', 'come', 'back', 'good', 'sound'])
    words = ' '.join(texts).lower()
    wordcloud = WordCloud(font_path='CabinSketch-Bold.ttf',
                          stopwords=my_stopwords,
                          background_color='black',
                          width=800,
                          height=600,
                          relative_scaling=1,
                          max_words=max_words
                         ).generate_from_text(words)
    print('Based on {0:,} texts'.format(len(texts)))
    
    fig, ax = plt.subplots(figsize=(15,10))
    ax.imshow(wordcloud)
    ax.axis('off')
    plt.show()

### Texts you've sent

In [ ]:
# Word cloud of the top 25 words I use based on the most recent 30,000 messages.

texts_from_me = fully_merged_messages_df[fully_merged_messages_df.is_from_me == 1].text[-30000:]
widgets.interact(
    generate_cloud,
    texts=widgets.fixed(texts_from_me),
    max_words=widgets.IntSlider(min=5,max=50,step=1,value=10, continuous_update=False,
                                description='Max words to show:'))

NameError: ignored

### Texts to/from a specific contact

In [ ]:
def _word_cloud_specific_contact(max_words, from_me, contact):
    contact = convert_unicode_to_str_if_needed(contact)
    if contact not in full_names:
        print('{} not found'.format(contact))
        return
    sliced_df = fully_merged_messages_df[(fully_merged_messages_df.full_name == contact) &
                                         (fully_merged_messages_df.is_from_me == from_me)].text
    generate_cloud(sliced_df, max_words)

widgets.interact(
    _word_cloud_specific_contact,
    max_words=widgets.IntSlider(min=5, max=50, step=1, value=10,
                                continuous_update=False, description='Max words to show:'),
    from_me=widgets.RadioButtons(
        options={'Show messages FROM me': True, 'Show messages TO me': False}, description=' '),
    contact=widgets.Text(value='Mom', description='Contact name:')
)